<H4>Load Libraries</H4>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import random
from scipy import ndimage
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

%matplotlib inline

from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json

Using Theano backend.


<H4>Load CIFAR-10 dataset</H4>

In [2]:
nb_classes = 10
 
def load_dataset():
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    print 'X_train shape:', X_train.shape
    print X_train.shape[0], 'train samples'
    print X_test.shape[0], 'test samples'
 
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
 
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
 
    return X_train, Y_train, X_test, Y_test

X_train, y_train, X_test, y_test = load_dataset()

X_train shape: (50000L, 3L, 32L, 32L)
50000 train samples
10000 test samples


<H4>Create a model with 4 convolutional layers and 2 fully-connected layers</H4>

In [3]:
def create_network(channels, image_rows, image_cols, lr, decay, momentum):
    model = Sequential()
 
    model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape = (channels, image_rows, image_cols)))
    model.add(Activation('relu'))
    model.add(Convolution2D(32, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Convolution2D(64, 3, 3, border_mode='valid'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    sgd = SGD(lr=lr, decay=decay, momentum=momentum, nesterov=True)
    
    model.compile(loss='categorical_crossentropy', optimizer=sgd)
    
    return model

model = create_network(3, 32, 32, 0.01, 1e-6, 0.9)

<H4>Defining call backs</H4>

In [4]:
earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
checkpoint = ModelCheckpoint(filepath = 'best_model_0718.hdf5', verbose = 1, save_best_only = True)

<H4>Fit the model with 10% validation split</H4>

In [5]:
model.fit(X_train, y_train, nb_epoch=100, batch_size=128, validation_split=0.1, verbose=1, callbacks=[checkpoint, earlystop])

Train on 45000 samples, validate on 5000 samples
Epoch 1/100
45000/45000 [==============================] - 2305s - loss: 2.0377 - val_loss: 1.8121
Epoch 2/100
45000/45000 [==============================] - 2374s - loss: 1.6529 - val_loss: 1.4445
Epoch 3/100
45000/45000 [==============================] - 2232s - loss: 1.4460 - val_loss: 1.3394
Epoch 4/100
45000/45000 [==============================] - 2232s - loss: 1.3232 - val_loss: 1.2194
Epoch 5/100
45000/45000 [==============================] - 2296s - loss: 1.2272 - val_loss: 1.0894
Epoch 6/100
45000/45000 [==============================] - 2317s - loss: 1.1510 - val_loss: 1.0094
Epoch 7/100
45000/45000 [==============================] - 2327s - loss: 1.0771 - val_loss: 0.9938
Epoch 8/100
45000/45000 [==============================] - 2314s - loss: 1.0204 - val_loss: 0.9132
Epoch 9/100
45000/45000 [==============================] - 2327s - loss: 0.9725 - val_loss: 0.9113
Epoch 10/100
45000/45000 [==============================] - 

<H4>Save the model and the weights</H4>

In [6]:
json_model = model.to_json()
with open("model_final.json", "w") as json_file:
    json_file.write(json_model)
    
model.save_weights('model_weights_final.h5')

<H4>Load the model and the weights saved using checkpoint</H4>

In [9]:
json_file = open('model_final.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model1 = model_from_json(loaded_model_json)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model1.compile(loss='categorical_crossentropy', optimizer=sgd)

model1.load_weights("best_model_0718.hdf5")

<H4>Verify the accuracy of the model on test set</H4>

In [11]:
test_loss = model1.evaluate(X_test, y_test, batch_size = 64, verbose=1)
print 'Test Loss:', test_loss

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Y_test = np.dot(y_test, classes).astype(int)

y_predict1 = model1.predict_classes(X_test, verbose = 1)
test_accuracy1 = accuracy_score(Y_test, y_predict1)
print 'Test Accuracy:', test_accuracy1 

10000/10000 [==============================] - 95s    
Test Loss: 0.638751012707
10000/10000 [==============================] - 102s   
Test Accuracy: 0.7786


In [12]:
test_loss = model.evaluate(X_test, y_test, batch_size = 64, verbose=1)
print 'Test Loss:', test_loss

y_predict = model.predict_classes(X_test, verbose = 1)
test_accuracy = accuracy_score(Y_test, y_predict)
print 'Test Accuracy:', test_accuracy

10000/10000 [==============================] - 97s    
Test Loss: 0.636110443497
10000/10000 [==============================] - 103s   
Test Accuracy: 0.7855
